In [54]:
# Pacotes
import pandas as pd    # manipulação de planilhas e data.frames
import os              # manipulação de arquivos
import shutil          # comprimir arquivos

In [55]:
# Função que pega as linhas do obj que tem a string de interesse e acrescenta var uf, mes, ano e var (eu criei)
def filtrar_linhas(obj, string, uf, mes, ano, var):
  obj_fin = obj[obj['var1'].str.contains(string, na = False)].dropna(axis = 'columns')

  obj_fin['uf'] = uf
  obj_fin['mes'] = mes
  obj_fin['ano'] = ano
  obj_fin['var'] = var

  if var == 'psico':
    obj_fin = obj_fin[obj_fin['var4'] == 'Total']

  return obj_fin

In [56]:
# Usaremos apenas os arquivos a partir de 2014
nome_diret = 'SISDEPEN_ARQS'

arqs = os.listdir(nome_diret)

arqs_usar = []

for arq in arqs:
  # Arqs cujo nome fogem ao padrão
  # Como são de 2020 vou usar e já crescento na lista arqs_usar
  if arq in ('prjun2020.xls', 'rnjun2020.xls'): 
    arqs_usar.append(arq)
    #print(arq)
  else: 
    ano = int(float(arq[7:11]))
    if ano >= 2014:
      arqs_usar.append(arq)

len(arqs_usar)

350

In [58]:
d = {'level': ['999'], 'value': [999], 'uf': ['999'], 'mes': ['999'], 'ano': [999], 'var': ['999']}
df_infogeral = pd.DataFrame(data = d)

e = {'level': ['999'], 'total': [999], 'porc': ['999'], 'uf': ['999'], 'mes': ['999'], 'ano': [999], 'var': ['999']}
df_sala = pd.DataFrame(data = e)

aux = {'level': ['999'], 'masc': ['999'], 'fem': ['999'], 'total': [999], 'uf': ['999'], 'mes': ['999'], 'ano': [999], 'var': ['999']}
df_cons = pd.DataFrame(data = aux)

aux2 = {'level': ['999'], 'masc': ['999'], 'fem': ['999'], 'misto': ['999'], 'total': [999], 'uf': ['999'], 'mes': ['999'], 'ano': [999], 'var': ['999']}
df_totestab = pd.DataFrame(data = aux2)

#var1	var4	var6	var8	var10	var12	var14
aux3 = {'level': ['999'], 'gen': ['999'], 'Efetivo': [999], 'Comissionado': [999], 'Terceirizado': [999], 'Temporário': [999], 'total': [999], 'uf': ['999'], 'mes': ['999'], 'ano': [999], 'var': ['999']}
df_psico = pd.DataFrame(data = aux3)

df_suic = pd.DataFrame(data = aux)

df_psico

,level,gen,Efetivo,Comissionado,Terceirizado,Temporário,total,uf,mes,ano,var
0,999,999,999,999,999,999,999,999,999,999,999


In [59]:
for arq in arqs_usar:
  # Criando variaveis a partir de slicing do nome do arq
  # Arqs cujo nome fogem ao padrão
  if arq in ('prjun2020.xls', 'rnjun2020.xls'):
    UF = arq[0:2]
    MES = arq[2:5]
    ANO = arq[5:9]
  else:
    UF = arq[0:2]
    MES = arq[3:6]
    ANO = arq[7:11]


  # Lendo os dados
  popcarcer = pd.read_excel(nome_diret + '/' + arq,   
                            skiprows = 7, 
                            names = ["var0", "var1", "var2", "var3", "var4", "var5",
                              "var6", "var7", "var8", "var9", "var10", "var11",
                              "var12", "var13", "var14", "var15", "var16", "var17",
                              "var18", "var19", "var20", "var21", "var22"],
                            index_col = 0,
                            header = None)
  
  # Excluir as linhas que são completamente de NA
  popcarcer = popcarcer.dropna(how = 'all')

  # Imputando as células mescladas com a ocorrência imediatamente anterior: 
  popcarcer['var1'] = popcarcer['var1'].fillna(method = 'pad') 
  
  # Info
  infogeral = filtrar_linhas(popcarcer, 'População carcerária', UF, MES, ANO, 'pop')
  infogeral = infogeral.rename(columns = {list(infogeral)[0]:'level', list(infogeral)[1]:'value'})
  df_infogeral = df_infogeral.append([infogeral])

  # Total de estab
  totestab = filtrar_linhas(popcarcer, 'Total de estabelecimentos penais', UF, MES, ANO, 'pop')
  totestab = totestab.rename(columns = {list(totestab)[0]:'level', list(totestab)[1]:'masc', list(totestab)[2]:'fem', list(totestab)[3]:'misto', list(totestab)[4]:'total'})
  df_totestab = df_totestab.append([totestab])

  # Sala de Atendimento Psicologico
  sala = filtrar_linhas(popcarcer, 'Leitos Psiquiátricos|sala exclusiva de atendimento psicológico|sala de atendimento psicológico compartilhada', UF, MES, ANO, 'sala')
  sala = sala.rename(columns = {list(sala)[0]:'level', list(sala)[1]:'total', list(sala)[2]:'porc'}) 
  df_sala = df_sala.append([sala])

  # Consultas
  cons = filtrar_linhas(popcarcer, 'Consultas psicológicas', UF, MES, ANO, 'cons')
  cons = cons.rename(columns = {list(cons)[0]:'level', list(cons)[1]:'masc', list(cons)[2]:'fem', list(cons)[3]:'total'}) 
  df_cons = df_cons.append([cons])

  # Suicidios
  suic = filtrar_linhas(popcarcer, 'Óbitos', UF, MES, ANO, 'suic')
  suic = suic.rename(columns = {list(suic)[0]:'level', list(suic)[1]:'masc', list(suic)[2]:'fem', list(suic)[3]:'total'}) 
  df_suic = df_suic.append([suic])  

  # Psicologos
  psico = filtrar_linhas(popcarcer, 'Terapeutas|Psicólogos|Psicólogos/as|Psiquiatras|psiquiatras', UF, MES, ANO, 'psico')
  psico = psico.rename(columns = {list(psico)[0]:'level', list(psico)[1]:'gen', 
                                  list(psico)[2]:'Efetivo', list(psico)[3]:'Comissionado',
                                  list(psico)[4]:'Terceirizado', list(psico)[5]:'Temporário',
                                  list(psico)[6]:'total'})

  df_psico = df_psico.append([psico]) 
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [61]:
# Função que exclui a linha inicial que eu criei, tira NAs e ordena as linhas
def ajeitar_ordenar_linhas(obj):
  obj_fin = obj.query(' uf != "999" ')
  obj_fin = obj_fin.dropna(axis = 'columns')
  obj_fin = obj_fin.sort_values(by = ['uf', 'ano', 'mes'])
  return obj_fin

In [62]:
df_infogeral = ajeitar_ordenar_linhas(df_infogeral)
df_totestab = ajeitar_ordenar_linhas(df_totestab)
df_sala = ajeitar_ordenar_linhas(df_sala)
df_cons = ajeitar_ordenar_linhas(df_cons)
df_suic = ajeitar_ordenar_linhas(df_suic)
df_psico = ajeitar_ordenar_linhas(df_psico)

In [ ]:
df_psico

In [66]:
# Gerando planilhas desses arqs
#os.mkdir('SISDEPEN_SELECTEDVAR')

df_infogeral.to_csv('SISDEPEN_SELECTEDVAR/df_infogeral.csv', index = False)
df_totestab.to_csv('SISDEPEN_SELECTEDVAR/df_totestab.csv', index = False)
df_sala.to_csv('SISDEPEN_SELECTEDVAR/df_sala.csv', index = False)
df_cons.to_csv('SISDEPEN_SELECTEDVAR/df_cons.csv', index = False)
df_suic.to_csv('SISDEPEN_SELECTEDVAR/df_suic.csv', index = False)
df_psico.to_csv('SISDEPEN_SELECTEDVAR/df_psico.csv', index = False)

In [67]:
# Comprimindo os arqs p fazer download
shutil.make_archive('SISDEPEN_SELECTEDVAR', 'zip', 'SISDEPEN_SELECTEDVAR')

'/content/SISDEPEN_SELECTEDVAR.zip'